In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Lambda, BatchNormalization, Flatten
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

In [3]:
X_train = np.load('../data/X_train.npy')
X_valid = np.load('../data/X_valid.npy')
y_train = np.load('../data/y_train.npy')
y_valid = np.load('../data/y_valid.npy')
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(36000, 784) (36000, 10)
(6000, 784) (6000, 10)


In [4]:
mean_px = X_train.mean()
std_px = X_train.std()

def normalize(x):
    return (x - mean_px) / std_px

### Model Architecture

In [5]:
model = Sequential()
model.add(BatchNormalization(input_shape=(28,28,1), axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [6]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
__________

In [8]:
params = {}
params['x'] = X_train.reshape(36000,28,28,1)
params['y'] = y_train
params['validation_data'] = (X_valid.reshape(6000,28,28,1), y_valid)

### Train Model

In [9]:
model.optimizer.lr = 0.001
params['epochs'] = 1
model.fit(**params)

Train on 36000 samples, validate on 6000 samples
Epoch 1/1
36000/36000 [==============================] - 90s - loss: 0.1622 - acc: 0.9523 - val_loss: 0.0718 - val_acc: 0.9758


In [10]:
model.optimizer.lr = 0.01
params['epochs'] = 3
model.fit(**params)

Train on 36000 samples, validate on 6000 samples
Epoch 1/3
36000/36000 [==============================] - 92s - loss: 0.0693 - acc: 0.9779 - val_loss: 0.0443 - val_acc: 0.9862
Epoch 2/3
36000/36000 [==============================] - 140s - loss: 0.0518 - acc: 0.9835 - val_loss: 0.0659 - val_acc: 0.9800
Epoch 3/3
36000/36000 [==============================] - 113s - loss: 0.0501 - acc: 0.9849 - val_loss: 0.0390 - val_acc: 0.9897


In [11]:
model.optimizer.lr = 0.005
params['epochs'] = 39
model.fit(**params)

Train on 36000 samples, validate on 6000 samples
Epoch 1/3
36000/36000 [==============================] - 89s - loss: 0.0445 - acc: 0.9863 - val_loss: 0.0455 - val_acc: 0.9865
Epoch 2/3
36000/36000 [==============================] - 89s - loss: 0.0346 - acc: 0.9888 - val_loss: 0.0468 - val_acc: 0.9882
Epoch 3/3
36000/36000 [==============================] - 89s - loss: 0.0326 - acc: 0.9895 - val_loss: 0.0481 - val_acc: 0.9887


In [12]:
model.save_weights('../model/model_1.h5')

### Score Model

In [13]:
X_score = np.load('../data/X_score.npy').reshape(28000,28,28,1)

In [14]:
predict = model.predict_classes(X_score)

27968/28000 [============================>.] - ETA: 0s

In [15]:
predict = pd.DataFrame(predict, columns=['Label'])
predict = predict.reset_index()
predict.columns = ['ImageId','Label']
predict['ImageId'] = predict['ImageId'] + 1

In [16]:
predict.to_csv('../data/submit_v1.csv', index=False)

In [17]:
### Submission scored 0.98942 on LB